In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import time

SYMBOL = 'AAPL'
INTERVAL = '5m'
POLLING_INTERVAL = 60  # seconds

def fetch_data():
    end_time = datetime.now()
    start_time = end_time - timedelta(days=1)
    df = yf.download(SYMBOL, start=start_time, end=end_time, interval=INTERVAL)
    return df

def analyze_trend(df):
    market_open_time = df.index[0].replace(hour=9, minute=30)
    market_45_min_time = market_open_time + timedelta(minutes=45)
    df_45_min = df.loc[(df.index >= market_open_time) & (df.index < market_45_min_time)]

    if df_45_min.empty:
        return "No data yet"

    highest_highs = df_45_min['High']
    lowest_lows = df_45_min['Low']

    if highest_highs.is_monotonic_increasing and lowest_lows.is_monotonic_increasing:
        trend = 'uptrend'
    elif highest_highs.is_monotonic_decreasing and lowest_lows.is_monotonic_decreasing:
        trend = 'downtrend'
    else:
        trend = 'sideways'

    return trend

while True:
    try:
        df = fetch_data()
        trend = analyze_trend(df)
        print(f'{datetime.now()}: Trend for the first 45 minutes: {trend}')
    except Exception as e:
        print(f'Error fetching or analyzing data: {e}')

    time.sleep(POLLING_INTERVAL)


In [ ]:
# list all stocks
url = "ftp://ftp.nasdaqtrader.com/SymbolDirectory/nasdaqlisted.txt"
df=pd.read_csv(url, sep="|")
print(df.head())
print(df['Symbol'].head())
print(len(df['Symbol']))
SYMBOLS=df.Symbol.tolist()
print(SYMBOLS)

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import time
import subprocess

# SYMBOLS = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA'] 
SYMBOLS = btc_symbols=pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].Symbol.tolist() # List of big movers, can be adjusted
# url = "ftp://ftp.nasdaqtrader.com/SymbolDirectory/nasdaqlisted.txt"
# df=pd.read_csv(url, sep="|")
INTERVAL = '1m'
POLLING_INTERVAL = 60  # seconds

def fetch_data(symbol):
    end_time = datetime.now()
    start_time = end_time - timedelta(days=1)
    df = yf.download(symbol, start=start_time, end=end_time, interval=INTERVAL)
    return df

def analyze_trend(df):
    market_open_time = df.index[0].replace(hour=9, minute=30)
    market_30_min_time = market_open_time + timedelta(minutes=30)
    df_30_min = df.loc[(df.index >= market_open_time) & (df.index < market_30_min_time)]

    if df_30_min.empty:
        return "No data yet"

    highest_highs = df_30_min['High']
    lowest_lows = df_30_min['Low']

    if highest_highs.is_monotonic_increasing and lowest_lows.is_monotonic_increasing:
        trend = 'uptrend'
    elif highest_highs.is_monotonic_decreasing and lowest_lows.is_monotonic_decreasing:
        trend = 'downtrend'
    else:
        trend = 'sideways'

    return trend

def send_notification(message):
    subprocess.run(['notify-send', message])

previous_trends = {symbol: None for symbol in SYMBOLS}

i=0
while True:
    print(f"SCAN N°{i+1}")
    for symbol in SYMBOLS:
        try:
            df = fetch_data(symbol)
            trend = analyze_trend(df)
            if trend != previous_trends[symbol] and trend in ['uptrend', 'downtrend']:
                send_notification(f'{symbol} is in an {trend}')
                previous_trends[symbol] = trend
            print(f'{datetime.now()}: {symbol} - Trend for the first 30 minutes: {trend}')
        except Exception as e:
            print(f'Error fetching or analyzing data: {e}')
    i=i+1
    time.sleep(POLLING_INTERVAL)


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

def get_nvda_data(start_date, end_date):
  """
  Fetches NVDA data from Yahoo Finance and calculates VWAP.

  Args:
    start_date: Start date for data.
    end_date: End date for data.

  Returns:
    Pandas DataFrame containing NVDA data with VWAP column.
  """

  # Fetch NVDA data
  nvda = yf.download("NVDA", start=start_date, end=end_date)

  # Calculate VWAP
  nvda['VWAP'] = (nvda['Close'] * nvda['Volume']).cumsum() / nvda['Volume'].cumsum()

  return nvda

# Example usage
start_date = "2023-01-01"
end_date = "2024-08-08"
nvda_data = get_nvda_data(start_date, end_date)
nvda_data.to_csv("/tmp/nvda_data.csv")
# Correlation analysis
correlation = nvda_data['Close'].corr(nvda_data['VWAP'])
print("Correlation between NVDA price and VWAP:", correlation)


In [ ]:
import yfinance as yf
import time
from datetime import datetime
import os

def send_notification(stock, current_price, vwap, daily_range):
    title = "Stock Alert"
    message = (
        f"Stock {stock} is trading near VWAP: {current_price} (VWAP: {vwap}) "
        f"and has a daily range of {daily_range}."
    )
    os.system(f'notify-send "{title}" "{message}"')

def calculate_vwap(data):
    # VWAP = sum(price * volume) / sum(volume)
    vwap = (data['Close'] * data['Volume']).sum() / data['Volume'].sum()
    return vwap

def check_stock_price(stock, threshold):
    try:
        # Fetch 1-minute interval data for the current day
        live_data = yf.download(tickers=stock, period='1d', interval='1m')
        if live_data.empty:
            return

        # Calculate VWAP based on 1-minute intervals for the day
        vwap = calculate_vwap(live_data)

        # Get the latest price
        current_price = live_data['Close'].iloc[-1]

        # Calculate the daily range
        daily_high = live_data['High'].max()
        daily_low = live_data['Low'].min()
        daily_range = daily_high - daily_low

        # Check if the current price is within +/- threshold % of VWAP
        # AND if the daily range is over $3
        if abs(current_price - vwap) / vwap <= threshold and daily_range > 3:
            send_notification(stock, current_price, vwap, daily_range)
    except Exception as e:
        print(f"Error checking stock {stock}: {e}")

def main(stocks, threshold=0.005):  # Default threshold is 0.5%
    while True:
        print(f"Checking stocks at {datetime.now()}...")
        for stock in stocks:
            check_stock_price(stock, threshold)
        time.sleep(60)  # Check every 60 seconds

if __name__ == "__main__":
    stocks = [
        "AAPL", "MSFT", "AMZN", "GOOGL", "GOOG", "FB", "TSLA", "BRK.B", "NVDA",
        "JPM", "JNJ", "V", "PG", "UNH", "HD", "MA", "DIS", "PYPL", "NFLX", "INTC",
        "VZ", "KO", "PFE", "MRK", "CSCO", "PEP", "ABT", "COST", "CMCSA", "XOM",
        "BAC", "WMT", "ADBE", "NKE", "T", "CRM", "MCD", "QCOM", "MDT", "LLY",
        "ORCL", "NEE", "UPS", "IBM", "TXN", "HON", "CVX", "BA", "WFC"
    ]  # List of important US stocks
    
    # You can adjust the threshold here, 0.005 represents 0.5%
    main(stocks, threshold=0.005)
